In [1]:
#!pip install --upgrade google-cloud-language

In [2]:
from google.colab import files
from google.cloud import storage
import sys 

keyfile_upload = files.upload()
keyfile = list(keyfile_upload.keys())[0]
%env GOOGLE_APPLICATION_CREDENTIALS $keyfile
! gcloud auth activate-service-account --key-file $keyfile

Saving linear-quasar-304321-e65f357ecf8a.json to linear-quasar-304321-e65f357ecf8a.json
env: GOOGLE_APPLICATION_CREDENTIALS=linear-quasar-304321-e65f357ecf8a.json
Activated service account credentials for: [nlpmay21@linear-quasar-304321.iam.gserviceaccount.com]


In [3]:
from pathlib import Path
import os 
import argparse
import re
import io, os
import six

In [8]:
from google.cloud import language_v1

def sample_analyze_entities(text_content):
  entity_type = []
  salience_score = []
  wikipedia_url = []
  mid = []
  mention_text = []
  mention_type = []
  client = language_v1.LanguageServiceClient()
  type_ = language_v1.Document.Type.PLAIN_TEXT
  language = "en"
  document = {"content": text_content, "type_": type_, "language": language}
  encoding_type = language_v1.EncodingType.UTF8
  response = client.analyze_entities(request = {'document': document, 'encoding_type': encoding_type})
  # Loop through entitites returned from the API
  for entity in response.entities:
      print(u"Representative name for the entity: {}".format(entity.name))
      # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
      print(u"Entity type: {}".format(language_v1.Entity.Type(entity.type_).name))
      # Get the salience score associated with the entity in the [0, 1.0] range
      print(u"Salience score: {}".format(entity.salience))
      # Loop over the metadata associated with entity. For many known entities,
      # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid
      # Some entity types may have additional metadata, e.g. ADDRESS entities
      # may have metadata for the address street_name, postal_code, et al.
      for metadata_name, metadata_value in entity.metadata.items():
          print(u"{}: {}".format(metadata_name, metadata_value))
          print('--------------------')
      # Loop over the mentions of this entity in the input document.
      # The API currently supports proper noun mentions.
      for mention in entity.mentions:
          print(u"Mention text: {}".format(mention.text.content))
          # Get the mention type, e.g. PROPER for proper noun
          print(
              u"Mention type: {}".format(language_v1.EntityMention.Type(mention.type_).name)
          )
          print("=============================")
  print(u"Language of the text: {}".format(response.language))

In [9]:
text = '''Norma Bouchard, an internationally recognized scholar and innovator with 
extensive experience leading diverse academic programs, has been selected as 
the new provost at Chapman University. Norma Bouchard, brings extensive 
experience as a researcher, scholar and leader to her new role as Chapman’s 
chief academic officer.
The announcement by Chapman President Daniele C. Struppa follows a 
nationwide search to replace Glenn Pfeiffer, who is retiring in July after 
five highly effective years as provost. Following a sabbatical, Pfeiffer 
will return to his previous position as a professor of accounting in Argyros 
School of Business and Economics.'''

In [10]:
sample_analyze_entities(text)

Representative name for the entity: Norma Bouchard
Entity type: PERSON
Salience score: 0.2968907356262207
mid: /m/0c3btwl
--------------------
Mention text: Norma Bouchard
Mention type: PROPER
Mention text: scholar
Mention type: COMMON
Mention text: Norma Bouchard
Mention type: PROPER
Representative name for the entity: experience
Entity type: OTHER
Salience score: 0.13601776957511902
Mention text: experience
Mention type: COMMON
Representative name for the entity: innovator
Entity type: PERSON
Salience score: 0.12034868448972702
Mention text: innovator
Mention type: COMMON
Representative name for the entity: Glenn Pfeiffer
Entity type: PERSON
Salience score: 0.09866064786911011
mid: /g/113sg8byv
--------------------
Mention text: Glenn Pfeiffer
Mention type: PROPER
Mention text: Pfeiffer
Mention type: PROPER
Representative name for the entity: provost
Entity type: PERSON
Salience score: 0.05343121662735939
Mention text: provost
Mention type: COMMON
Representative name for the entity: 

In [15]:
from google.cloud import language_v1
def sample_analyze_syntax(text_content):
    client = language_v1.LanguageServiceClient()
    # text_content = 'This is a short sentence.'
    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT
    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}
    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8
    response = client.analyze_syntax(request = {'document': document, 'encoding_type': encoding_type})
    # Loop through tokens returned from the API
    for token in response.tokens:
        # Get the text content of this token. Usually a word or punctuation.
        text = token.text
        print('-------------------------------------')
        print(u"Token text: {}".format(text.content))
        print(
            u"Location of this token in overall document: {}".format(text.begin_offset)
        )
        # Get the part of speech information for this token.
        # Parts of spech are as defined in:
        # http://www.lrec-conf.org/proceedings/lrec2012/pdf/274_Paper.pdf
        part_of_speech = token.part_of_speech
        # Get the tag, e.g. NOUN, ADJ for Adjective, et al.
        print(
            u"Part of Speech tag: {}".format(
                language_v1.PartOfSpeech.Tag(part_of_speech.tag).name
            )
        )
        # Get the voice, e.g. ACTIVE or PASSIVE
        print(u"Voice: {}".format(language_v1.PartOfSpeech.Voice(part_of_speech.voice).name))
        # Get the tense, e.g. PAST, FUTURE, PRESENT, et al.
        print(u"Tense: {}".format(language_v1.PartOfSpeech.Tense(part_of_speech.tense).name))
        # See API reference for additional Part of Speech information available
        # Get the lemma of the token. Wikipedia lemma description
        # https://en.wikipedia.org/wiki/Lemma_(morphology)
        print(u"Lemma: {}".format(token.lemma))
        # Get the dependency tree parse information for this token.
        # For more information on dependency labels:
        # http://www.aclweb.org/anthology/P13-2017
        dependency_edge = token.dependency_edge
        print(u"Head token index: {}".format(dependency_edge.head_token_index))
        print(
            u"Label: {}".format(language_v1.DependencyEdge.Label(dependency_edge.label).name)
        )

In [16]:
sample_analyze_syntax(text)

-------------------------------------
Token text: Norma
Location of this token in overall document: 0
Part of Speech tag: NOUN
Voice: VOICE_UNKNOWN
Tense: TENSE_UNKNOWN
Lemma: Norma
Head token index: 1
Label: NN
-------------------------------------
Token text: Bouchard
Location of this token in overall document: 6
Part of Speech tag: NOUN
Voice: VOICE_UNKNOWN
Tense: TENSE_UNKNOWN
Lemma: Bouchard
Head token index: 19
Label: NSUBJPASS
-------------------------------------
Token text: ,
Location of this token in overall document: 14
Part of Speech tag: PUNCT
Voice: VOICE_UNKNOWN
Tense: TENSE_UNKNOWN
Lemma: ,
Head token index: 1
Label: P
-------------------------------------
Token text: an
Location of this token in overall document: 16
Part of Speech tag: DET
Voice: VOICE_UNKNOWN
Tense: TENSE_UNKNOWN
Lemma: an
Head token index: 6
Label: DET
-------------------------------------
Token text: internationally
Location of this token in overall document: 19
Part of Speech tag: ADV
Voice: VOICE_

In [17]:
def sample_classify_text(text_content):
    client = language_v1.LanguageServiceClient()
    # text_content = 'That actor on TV makes movies in Hollywood and also 
    # stars in a variety of popular new TV shows.'
    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}
    response = client.classify_text(request = {'document': document})
    # Loop through classified categories returned from the API
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        print(u"Category name: {}".format(category.name))
        # Get the confidence. Number representing how certain the classifier
        # is that this category represents the provided text.
        print(u"Confidence: {}".format(category.confidence))

In [18]:
sample_classify_text(text)

Category name: /Jobs & Education/Education/Colleges & Universities
Confidence: 0.6299999952316284
